# Download necessary Libraries

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
!pip install wandb

# Download model & Tokenizer

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "silma-ai/SILMA-9B-Instruct-v1.0",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("silma-ai/SILMA-9B-Instruct-v1.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.91G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

## Lora adaptation

In [3]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8945664 || all params: 9250651648 || trainable%: 0.09670306850149374


# Data loadding & preprocessing (cooking 🍳)

## Data loadding

In [ ]:
from datasets import load_dataset

dataset = load_dataset("faisalq/poemsDataset")

README.md:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

poemsDataset.csv:   0%|          | 0.00/823M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2090907 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
import json
import pandas as pd

dataset_path = "filtered_clean.jsonl"
df = pd.read_json(dataset_path, lines=True)

df.head()


,title,body,meter,rhyme,link
0,بشائر نعمى لا يقوم لها شكر,بشائر نعمى لا يقوم لها شكر<|SHATR|>وتجديد فخر ...,طويل,ر,https://poetry.dctabudhabi.ae/#/poems/129384-%...
1,تعاميت في دين الهَوى فَكَأَنَّني,تعاميت في دين الهوى فكأنني<|SHATR|>ضرير قد است...,طويل,ت,https://poetry.dctabudhabi.ae/#/poems/86664-%D...
2,حكى مِشيةَ المخمورِ من غير قُدْرة,حكى مشية المخمور من غير قدرة<|SHATR|>على مزلئج...,طويل,ر,https://poetry.dctabudhabi.ae/#/poems/80164-%D...
3,وأبدت لردّ الليل سود ضفائر,وأبدت لرد الليل سود ضفائر<|SHATR|>فضاء محياها ...,طويل,د,https://poetry.dctabudhabi.ae/#/poems/100696-%...
4,تَقُولُ ابْنَةُ الْعَبْسِيِّ قَدْ شِبْتَ بَعْدَنا,تقول ابنة العبسي قد شبت بعدنا<|SHATR|>وكل امرئ...,طويل,ب,https://poetry.dctabudhabi.ae/#/poems/21183-%D...


## Data cleaning

## prompt template

In [ ]:
# Define EOS token
EOS_TOKEN = tokenizer.eos_token

In [ ]:
PROMPT_TEMPLATE = """### تعليمات:
قم بإنشاء قصيدة عربية بناءً على الموضوع، البحر، والقافية المحددة.

### المدخلات:
- **الموضوع:** {poem_title}
- **البحر:** {meter}
- **القافية:** {rhyme}

### الاستجابة:
{first_hemistich} {second_hemistich}""" + EOS_TOKEN

In [6]:
import json
from datasets import Dataset

# Load dataset
dataset_path = "filtered_clean.jsonl"
data = []
with open(dataset_path, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

# Define EOS token
EOS_TOKEN = tokenizer.eos_token

# Function to format dataset
def format_poetry_with_special_tokens(data):
    formatted_texts = []

    for entry in data:
        poem_body = entry["body"].strip()  # Keep <|SHATR|> and <|BEET|> unchanged

        text = f"""### تعليمات:
قم بإنشاء قصيدة عربية بناءً على الموضوع، البحر، والقافية المحددة.

### المدخلات:
- **الموضوع:** {entry['title']}
- **البحر:** {entry['meter']}
- **القافية:** {entry['rhyme']}

### الاستجابة:
{poem_body}""" + EOS_TOKEN

        formatted_texts.append({"text": text})

    return formatted_texts

formatted_data = format_poetry_with_special_tokens(data)
print(formatted_data[:1])  # Verify output

# Convert into Hugging Face dataset
dataset = Dataset.from_list(formatted_data)


[{'text': '### تعليمات:\nقم بإنشاء قصيدة عربية بناءً على الموضوع، البحر، والقافية المحددة.\n\n### المدخلات:\n- **الموضوع:** بشائر نعمى لا يقوم لها شكر\n- **البحر:** طويل\n- **القافية:** ر\n\n### الاستجابة:\nبشائر نعمى لا يقوم لها شكر<|SHATR|>وتجديد فخر لا يقابله فخر<|SHATR|><|BEET|>وصارم ملك قلدوا فيه صارما<|SHATR|>متى ما سما للفتح جوده النصر<|SHATR|><|BEET|>تسربل داود الوزير مطارفا<|SHATR|>وما هي إلا الحكم والنهي والأمر<|SHATR|><|BEET|>مطارف حاقانية النسج لوتها<|SHATR|>له الأبيض الفتاك لا البيض والصفر<|SHATR|><|BEET|>اذا قابلتها الشمس صدت كأنها<|SHATR|>فتاة رأت ضراء أعوزها الخدر<|SHATR|><|BEET|>متى اشرقت في مكبيه إستفزها<|SHATR|>لهيبته واعتدها الخوف والذعر<|SHATR|><|BEET|>مناكب لا تهتز إلا لسؤدد<|SHATR|>وكم منكب بالؤم قد هزه الغدر<|SHATR|><|BEET|>اذا نشرت يوما مساعيه للعلى<|SHATR|>بدا نشرها والعود زينه النشر<|SHATR|><|BEET|>اياد إذا مافاض زاخر نيلها<|SHATR|>لذي عثرة يستصغر الغيث والبحر<|SHATR|><|BEET|>تدر شطور العفو عن ذي جريرة<|SHATR|>وللظلم ضرع لا يدر له شطر<|SHATR|><|BEET|>وأوسع عد

In [ ]:
train_dataset = dataset.map(format_poetry, batched=True, remove_columns=dataset["train"].column_names)

print(train_dataset["train"][0]["text"])

Map:   0%|          | 0/442763 [00:00<?, ? examples/s]

### تعليمات:
قم بإنشاء قصيدة عربية بناءً على الموضوع، البحر، والقافية المحددة.

### المدخلات:
- **الموضوع:** عَفـا جـانِبُ الْبَطْحاءِ مِنْ إِبْنِ هاشِمٍ
- **البحر:** طويل
- **القافية:** م

### الاستجابة:
عفا جانب البطحاء من إبن هاشم وجاور لحدا خارجا في الغماغم
دعته المنايا دعوة فأجابها وما تركت في الناس مثل ابن هاشم
عشية راحوا يحملون سريره تعاوره أصحابه في التزاحم
فإن يك غالته المنايا وريبها فقد كان معطاء كثير التراحم <eos>


In [ ]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 38301
    })
})

## Tokenization

In [7]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Check tokenized output
print(tokenizer.convert_ids_to_tokens(tokenized_dataset[0]["input_ids"]))


Map:   0%|          | 0/35556 [00:00<?, ? examples/s]

['<bos>', '###', '▁تع', 'ليم', 'ات', ':', '\n', 'قم', '▁ب', 'إن', 'شاء', '▁قص', 'يدة', '▁ع', 'ربية', '▁بناء', 'ً', '▁على', '▁الموضوع', '،', '▁البحر', '،', '▁والق', 'افية', '▁الم', 'حد', 'دة', '.', '\n\n', '###', '▁المد', 'خل', 'ات', ':', '\n', '-', '▁**', 'الم', 'وضوع', ':**', '▁بش', 'ائر', '▁نعم', 'ى', '▁لا', '▁يقوم', '▁لها', '▁ش', 'كر', '\n', '-', '▁**', 'الب', 'حر', ':**', '▁ط', 'ويل', '\n', '-', '▁**', 'الق', 'افية', ':**', '▁ر', '\n\n', '###', '▁الاست', 'جابة', ':', '\n', 'بش', 'ائر', '▁نعم', 'ى', '▁لا', '▁يقوم', '▁لها', '▁ش', 'كر', '<', '|', 'SH', 'ATR', '|>', 'وت', 'جد', 'يد', '▁ف', 'خر', '▁لا', '▁يق', 'اب', 'له', '▁ف', 'خر', '<', '|', 'SH', 'ATR', '|', '><', '|', 'BE', 'ET', '|>', 'وص', 'ارم', '▁ملك', '▁ق', 'لد', 'وا', '▁فيه', '▁ص', 'ار', 'ما', '<', '|', 'SH', 'ATR', '|>', 'مت', 'ى', '▁ما', '▁سما', '▁لل', 'فتح', '▁ج', 'وده', '▁الن', 'صر', '<', '|', 'SH', 'ATR', '|', '><', '|', 'BE', 'ET', '|>', 'تس', 'رب', 'ل', '▁دا', 'ود', '▁الوز', 'ير', '▁مط', 'ار', 'فا', '<', '|', 'SH', 'ATR

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
#UUUUUUUUUUUUUU

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Apply tokenization to the dataset
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/38301 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 38301
    })
})

# Training

In [1]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [10]:
import os

checkpoint_dir = "/content/drive/My Drive/poetry_finetuning_checkpoints"

# Ensure the directory exists
os.makedirs(checkpoint_dir, exist_ok=True)


In [13]:
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hatimalhomid (hatimalhomid-education-com) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/poetry_finetuning_checkpoints",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=500,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=5,
    report_to="wandb",  # Enable WandB logging
    push_to_hub=True,
    do_train=True
)


In [15]:
wandb.init(project="poetry_finetuning", name="run_1")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
import os
hf_token = ''
os.environ["HF_TOKEN"] = hf_token


In [18]:
from transformers import Trainer, DataCollatorForLanguageModeling

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
50,3.031300
100,2.267600
150,2.034200
200,1.980800
250,1.971000
300,1.923800
350,1.895200
400,1.913500
450,1.862400
500,1.886200


TrainOutput(global_step=4444, training_loss=1.6916720839259218, metrics={'train_runtime': 24974.3882, 'train_samples_per_second': 2.847, 'train_steps_per_second': 0.178, 'total_flos': 1.8199146473214444e+18, 'train_loss': 1.6916720839259218, 'epoch': 1.999437507031162})

# push trained model to huggingface

In [ ]:
model.push_to_hub("hatim00101/Silma-fine-tuned-arabic-petry_v2.0_full-Tawil", # username_inHuggingFace/any name
                  token = hf_token,
                  commit_message="basic training test",
                  private=True)

adapter_model.safetensors:   0%|          | 0.00/35.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hatim00101/Silma-fine-tuned-arabic-petry_v2.0_full-Tawil/commit/9f52126250c53e38465e2f264bd229eb20900baa', commit_message='basic training test', commit_description='', oid='9f52126250c53e38465e2f264bd229eb20900baa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hatim00101/Silma-fine-tuned-arabic-petry_v2.0_full-Tawil', endpoint='https://huggingface.co', repo_type='model', repo_id='hatim00101/Silma-fine-tuned-arabic-petry_v2.0_full-Tawil'), pr_revision=None, pr_num=None)

#Load trained model

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "hatim00101/Silma-fine-tuned-arabic-petry_v2.0_full"
config = PeftConfig.from_pretrained(peft_model_id , token = hf_token)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

#Inference

In [ ]:
test_prompt = ''
batch = tokenizer("Test!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))